In [1200]:
import numpy as np
import pandas as pd
import random
from IPython.display import display


#### Load the matrices

In [1201]:
battery_expenditure = pd.read_csv('battery_expenditure_matrix.csv')
distance_matrix = pd.read_csv('distance_matrix.csv')
time_matrix = pd.read_csv('time_matrix.csv')

In [1202]:
customer_demands = [[3,4],[5,15],[3,7],[8,10],[6,13]]
# customer_demands = [[3,4],[5,15],[3,7],[8,7],[6,13]]
# customer_demands = [[3,15],[5,15],[3,15],[8,15],[6,15]]

In [1203]:
covered_nodes = []

for customer_demand in customer_demands:
    covered_nodes.append(customer_demand[0])
    covered_nodes.append(customer_demand[1])
    sorted(covered_nodes)


#### Generating random combination

In [1204]:
num_sequences=20
start_node=0
end_node=16
battery_threshold=10

w1=0.5
w2=0.5

In [1205]:
def carCapacityMaintained(route):
    curWt = 0
    for i in range(len(route)):
        wt = route[i][1]
        curWt += wt
        if curWt > 3:
            return False
        if curWt < 0:
            return False
         
    return True


In [1206]:
def allCustomersDemand(sequence):
    route=[]
    for node in sequence[1:-1]:
        route.append(node[0])
    return sorted(route) == sorted(covered_nodes)

In [1207]:
def sequenceIsValid(route):
    # check for validity if every customer reaches destination
    if not allCustomersDemand(route):
        return False

    # check for validity if car strength not increase 3
    if not carCapacityMaintained(route):
        return False
    
    #check if battery level not decrease less than 0
    for i in range(len(route) - 1):
        node1 = route[i][0]
        node2 = route[i + 1][0]
                
        if node1 == 1 or node1 == 2:
            battery_level=78

            battery_consumption = battery_expenditure.iloc[node1, node2]
            battery_level -= battery_consumption
            if battery_level < battery_threshold:
                return False
    return True

In [1208]:
def shuffle_array_except_first_last(arr):
    first_element = arr[0]
    last_element = arr[-1]
    internal_elements = arr[1:-1]
    random.shuffle(internal_elements)
    shuffled_array = [first_element] + internal_elements + [last_element]
    return shuffled_array

In [1209]:
def generate_random_sequences(num_sequences,customer_demands=customer_demands):
    sequences = []

    for _ in range(num_sequences):
        while True:
            route = [(0, 0) for _ in range(12)]
            used_positions = set()
            unused_positions = set()
            for i in range(12):
                unused_positions.add(i)
            unused_positions.remove(0)
            unused_positions.remove(11)
            used_positions.add(0)
            used_positions.add(11)

            for demand in customer_demands:
                position1 = random.randint(min(unused_positions),max(unused_positions)-1)
                while position1 in used_positions:
                    position1 = random.randint(min(unused_positions),max(unused_positions)-1)
                unused_positions.remove(position1)
                used_positions.add(position1)

                position2 = random.randint(position1 + 1, max(unused_positions))
                while position2 in used_positions:
                    position2 = random.randint(position1 + 1, max(unused_positions))
                unused_positions.remove(position2)
                used_positions.add(position2)

                route[position1] = (demand[0], +1)  # Insert the start node of the customer demand
                route[position2] = (demand[1], -1)  # Insert the end node of the customer demand

            
            if sequenceIsValid(route):
                sequences.append(route)
                break

    return sequences

In [1210]:
random_seq=generate_random_sequences(num_sequences)
random_seq

[[(0, 0),
  (6, 1),
  (5, 1),
  (3, 1),
  (13, -1),
  (3, 1),
  (7, -1),
  (8, 1),
  (15, -1),
  (10, -1),
  (4, -1),
  (0, 0)],
 [(0, 0),
  (6, 1),
  (3, 1),
  (4, -1),
  (3, 1),
  (8, 1),
  (13, -1),
  (5, 1),
  (15, -1),
  (7, -1),
  (10, -1),
  (0, 0)],
 [(0, 0),
  (3, 1),
  (6, 1),
  (13, -1),
  (4, -1),
  (3, 1),
  (5, 1),
  (15, -1),
  (8, 1),
  (10, -1),
  (7, -1),
  (0, 0)],
 [(0, 0),
  (8, 1),
  (6, 1),
  (10, -1),
  (13, -1),
  (5, 1),
  (3, 1),
  (3, 1),
  (15, -1),
  (7, -1),
  (4, -1),
  (0, 0)],
 [(0, 0),
  (6, 1),
  (13, -1),
  (8, 1),
  (3, 1),
  (10, -1),
  (7, -1),
  (3, 1),
  (4, -1),
  (5, 1),
  (15, -1),
  (0, 0)],
 [(0, 0),
  (8, 1),
  (5, 1),
  (6, 1),
  (15, -1),
  (3, 1),
  (7, -1),
  (10, -1),
  (3, 1),
  (13, -1),
  (4, -1),
  (0, 0)],
 [(0, 0),
  (6, 1),
  (8, 1),
  (3, 1),
  (13, -1),
  (4, -1),
  (3, 1),
  (10, -1),
  (7, -1),
  (5, 1),
  (15, -1),
  (0, 0)],
 [(0, 0),
  (5, 1),
  (6, 1),
  (13, -1),
  (3, 1),
  (3, 1),
  (4, -1),
  (8, 1),
  (7, -1),
  (

In [1211]:
def calculate_cost(sequence, time_matrix, distance_matrix, w1, w2):
    cost1 = 0
    cost2 = 0
    cost = 0

    for i in range(len(sequence) - 1):
        node1 = sequence[i][0]
        node2 = sequence[i + 1][0]

        cost1 += time_matrix.iloc[node1, node2]
        cost2 += distance_matrix.iloc[node1, node2]

        cost += (w1 * time_matrix.iloc[node1, node2]) + (w2 * distance_matrix.iloc[node1, node2])

    return pd.DataFrame({'Σt': [cost1], 'Σd': [cost2], 'w1*Σt+w2*Σd': [cost]})

In [1212]:
cost_for_each_sequence = [calculate_cost(sequence, time_matrix, distance_matrix, w1, w2) for sequence in random_seq]
result_df = pd.concat(cost_for_each_sequence, ignore_index=True)

In [1213]:
result_df

,Σt,Σd,w1*Σt+w2*Σd
0,3.430000,164.3514,83.890700
1,3.496778,166.5990,85.047889
2,3.145389,144.7588,73.952094
3,3.715944,180.1945,91.955222
4,3.532639,163.9634,83.748019
5,3.634917,172.1258,87.880358
6,3.533417,165.8738,84.703608
7,3.376556,164.5973,83.986928
8,3.584444,171.3188,87.451622
9,3.397000,161.0386,82.217800


## Genetic Algorithm implementation

In [1214]:
def initialize_population(population_size):
    population = generate_random_sequences(population_size)
    return population


In [1215]:
def tournament_selection(population, fitness_values, tournament_size):
    selected_parents = []
    for _ in range(len(population)):
        tournament_indices = np.random.choice(len(population), tournament_size, replace=False)
        tournament_fitness = [fitness_values[i] for i in tournament_indices]
        winner_index = tournament_indices[np.argmin(tournament_fitness)]
        selected_parents.append(population[winner_index])
    return selected_parents

In [1216]:
def cyclic_shift(lst, positions):
    length = len(lst)
    positions = positions % length  # Ensure positions is within the range of list length
    return lst[-positions:] + lst[:-positions]

In [1217]:
import random
#Redo
def ordered_crossover(parent1, parent2):
    # Select two random crossover points

    parent1= parent1[1:-1]
    parent2= parent2[1:-1]

    crossover_points = sorted(random.sample(range(1, len(parent1)), 2))
    while True:
        offspring = parent2[crossover_points[0]:crossover_points[1]]
        remaining =[]
        for i in parent1[crossover_points[1]:]:
            if offspring.count(i) + remaining.count(i) < parent1.count(i):
                remaining.append(i)
        
        
        for i in parent1[0:crossover_points[1]]:
            if offspring.count(i) + remaining.count(i) < parent1.count(i):
                remaining.append(i)
        
        offspring = offspring + remaining
        offspring = cyclic_shift(offspring,crossover_points[0])

        offspring= [(0,0)] + offspring + [(0,0)]
        if sequenceIsValid(offspring):
            break
        
        crossover_points = sorted(random.sample(range(1, len(parent1)), 2))

    parent1= [(0,0)] + parent1 + [(0,0)]
    parent2= [(0,0)] + parent2 + [(0,0)]
    return offspring


In [1218]:
def inversion_mutation(sequence):
    randPnts = sorted(random.sample(range(1, len(sequence)-2), 2))
    # print("Org: ",sequence)
    while True:
        start = randPnts[0]
        end = randPnts[1]
        
        mutated_sequence = sequence[:start] + list(reversed(sequence[start:end+1])) + sequence[end+1:]
        if sequenceIsValid(mutated_sequence):
            break
        randPnts = sorted(random.sample(range(1, len(sequence)-2), 2))
    # print("Mut: ",mutated_sequence)
    return mutated_sequence

In [1219]:
import random
def genetic_algorithm(num_generations, population_size, tournament_size,crossover_probability ,mutation_probability):
    population = initialize_population(population_size)
    # display(pd.DataFrame(population))
    for generation in range(num_generations):
        fitness_values = [calculate_cost(sequence, time_matrix, distance_matrix, w1, w2)['w1*Σt+w2*Σd'].values[0] for sequence in population]

        parents = tournament_selection(population, fitness_values, tournament_size)
        
        random.shuffle(parents)

        offspring = []

        for i in range(0, len(parents), 2):
            if i + 1 < len(parents):
                if random.uniform(0, 1) < crossover_probability:
                    child1 = ordered_crossover(parents[i], parents[i + 1])
                    child2 = ordered_crossover(parents[i + 1], parents[i])
                else:
                    child1 = parents[i]
                    child2 = parents[i + 1]
                    # print(child1)
                    # offspring.extend([child1, child2])
                offspring.append(child1)
                offspring.append(child2)


        mutated_offspring = []
        for child in offspring:
            if random.uniform(0, 1) < mutation_probability:
                mutated_child = inversion_mutation(child)
                mutated_offspring.append(mutated_child)
            else:
                mutated_offspring.append(child)

        population = mutated_offspring

    best_sequence = min(population, key=lambda x: calculate_cost(x, time_matrix, distance_matrix, w1, w2)['w1*Σt+w2*Σd'].values[0])
    
    return best_sequence,population


In [1220]:

best_sequence,final_population = genetic_algorithm(num_generations=100, population_size=50, tournament_size=6, crossover_probability=0.95,mutation_probability=0.3)
print("Best Sequence:", best_sequence)
costdf = calculate_cost(best_sequence, time_matrix, distance_matrix, w1, w2)
display(costdf)
print("Best Cost:",costdf.iloc[-1,-1])

Best Sequence: [(0, 0), (6, 1), (3, 1), (3, 1), (4, -1), (5, 1), (10, -1), (7, -1), (8, 1), (13, -1), (15, -1), (0, 0)]


,Σt,Σd,w1*Σt+w2*Σd
0,2.386028,104.9839,53.684964


Best Cost: 53.68496388888889


In [1221]:
cost_for_each_sequence = [calculate_cost(sequence, time_matrix, distance_matrix, w1, w2) for sequence in final_population]
result_df = pd.concat(cost_for_each_sequence, ignore_index=True)

In [1222]:
display(pd.DataFrame(final_population))

,0,1,2,3,4,5,6,7,8,9,10,11
0,"(0, 0)","(6, 1)","(3, 1)","(3, 1)","(4, -1)","(5, 1)","(10, -1)","(7, -1)","(8, 1)","(13, -1)","(15, -1)","(0, 0)"
1,"(0, 0)","(3, 1)","(6, 1)","(3, 1)","(4, -1)","(5, 1)","(10, -1)","(7, -1)","(8, 1)","(13, -1)","(15, -1)","(0, 0)"
2,"(0, 0)","(6, 1)","(3, 1)","(3, 1)","(4, -1)","(5, 1)","(10, -1)","(7, -1)","(8, 1)","(13, -1)","(15, -1)","(0, 0)"
3,"(0, 0)","(6, 1)","(3, 1)","(4, -1)","(3, 1)","(5, 1)","(10, -1)","(7, -1)","(8, 1)","(13, -1)","(15, -1)","(0, 0)"
4,"(0, 0)","(6, 1)","(3, 1)","(3, 1)","(4, -1)","(5, 1)","(10, -1)","(7, -1)","(8, 1)","(13, -1)","(15, -1)","(0, 0)"
5,"(0, 0)","(6, 1)","(3, 1)","(3, 1)","(4, -1)","(5, 1)","(10, -1)","(7, -1)","(8, 1)","(13, -1)","(15, -1)","(0, 0)"
6,"(0, 0)","(6, 1)","(3, 1)","(3, 1)","(4, -1)","(5, 1)","(10, -1)","(7, -1)","(8, 1)","(13, -1)","(15, -1)","(0, 0)"
7,"(0, 0)","(6, 1)","(3, 1)","(3, 1)","(4, -1)","(5, 1)","(10, -1)","(7, -1)","(8, 1)","(13, -1)","(15, -1)","(0, 0)"
8,"(0, 0)","(6, 1)","(8, 1)","(7, -1)","(10, -1)","(5, 1)","(4, -1)","(3, 1)","(3, 1)","(13, -1)","(15, -1)","(0, 0)"
9,"(0, 0)","(6, 1)","(3, 1)","(3, 1)","(4, -1)","(5, 1)","(10, -1)","(7, -1)","(8, 1)","(13, -1)","(15, -1)","(0, 0)"


In [1223]:
result_df

,Σt,Σd,w1*Σt+w2*Σd
0,2.386028,104.9839,53.684964
1,2.521667,112.0569,57.289283
2,2.386028,104.9839,53.684964
3,2.516361,111.2827,56.899531
4,2.386028,104.9839,53.684964
5,2.386028,104.9839,53.684964
6,2.386028,104.9839,53.684964
7,2.386028,104.9839,53.684964
8,2.620639,118.7067,60.663669
9,2.386028,104.9839,53.684964
